# VQE (Variational Quantum Eigensolver) — single-qubit worked example (Qiskit)

This notebook implements VQE for the Hamiltonian

\[ H = 2 Z + X + I \]

It includes:

- exact diagonalization (classical) to get the true ground-state energy
- a variational ansatz and a VQE run using Qiskit Aer simulator
- comparison of the VQE result against the exact value

**Requirements:** Qiskit installed (`pip install qiskit`), and Qiskit Aer for the statevector simulator.

Run the cells in order in a Jupyter environment where Qiskit is available.

## 1) Imports and setup

We'll import Qiskit utilities, build a quantum instance for reproducible simulation, and set seeds. If you don't have Qiskit installed, install it first with `pip install qiskit`.

In [ ]:
# 1) Imports and basic setup for Qiskit 1.0+
from qiskit_aer import AerSimulator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA, SPSA
from qiskit_algorithms.utils import algorithm_globals

# Import the Estimator: the new primitive for expectation value calculation
# Note: We use the Aer version for simulation here.
from qiskit_aer.primitives import Estimator

import numpy as np
import matplotlib.pyplot as plt

# For reproducibility
seed = 42
algorithm_globals.random_seed = seed

# Choose backend (We now configure the simulator through the Estimator)
# The 'method' is set in the Estimator instead of the QuantumInstance
simulator = AerSimulator()

# Print Qiskit version for confirmation
print('Setup complete. Qiskit version:', end=' ')
try:
    import qiskit
    print(qiskit.__version__)
except Exception as e:
    print('Qiskit import failed — make sure Qiskit is installed. Error:', e)


## 2) Build the Hamiltonian

We'll represent the Hamiltonian as a Qiskit `PauliSumOp`:

\[ H = 2 Z + X + I \]

This is a single-qubit operator; we can diagonalize the 2×2 matrix classically to obtain the exact eigenvalues for verification.

In [ ]:
# 2) Build the Hamiltonian H = 2*Z + X + I
# Using SparsePauliOp with Pauli string literals
H_op = SparsePauliOp.from_list([
    ("Z", 2.0),  # Represents 2 * Z
    ("X", 1.0),  # Represents 1 * X
    ("I", 1.0)   # Represents 1 * I
])

print('Hamiltonian (SparsePauliOp):')
print(H_op)

## 3) Exact classical solution

We diagonalize the 2×2 matrix to obtain exact eigenvalues and eigenvectors. The lowest eigenvalue is the true ground-state energy.

In [ ]:
# 3) Exact classical diagonalization
H_mat = H_op.to_matrix()
eigvals, eigvecs = np.linalg.eigh(H_mat)
print('Exact eigenvalues (lowest is ground-state energy):\n', eigvals)

ground_energy_exact = np.min(eigvals)
print(f'Exact ground-state energy = {ground_energy_exact:.12f}')

## 4) Variational ansatz and VQE setup

We pick a simple single-qubit variational form — `RealAmplitudes` with one repetition is sufficient for a single qubit. Then we create a `VQE` instance with a classical optimizer (COBYLA by default).

In [ ]:
# 4) VQE setup
ansatz = RealAmplitudes(num_qubits=1, reps=1)

# Choose an optimizer - COBYLA works well for small noiseless runs
optimizer = COBYLA(maxiter=200)

# Initialize the ESTIMATOR V2 primitive for the Aer simulator
# This is the correct import for the new primitives interface
from qiskit_aer.primitives import Estimator as AerEstimator

estimator = AerEstimator(
    backend_options={
        'method': 'statevector'  # Ensures we use the exact statevector method
    },
    run_options={
        'seed': seed  # For reproducibility
    },
    # This is key for V2: set the number of shots to None for exact statevector calculation
    transpile_options={"seed_transpiler": seed}
)

# Build VQE instance using the Estimator
vqe = VQE(estimator, ansatz, optimizer)

print('Ansatz:') 
print(ansatz)

## 5) Run VQE

We run VQE to estimate the ground-state energy and then compare it with the exact value.

In [ ]:
# 5) Run VQE
# The operator is now the SparsePauliOp (H_op) we created earlier
result = vqe.compute_minimum_eigenvalue(operator=H_op)

print('\nVQE result summary:')
print(result)

vqe_energy = result.eigenvalue.real
print(f'\nVQE estimated ground-state energy = {vqe_energy:.12f}')
print(f'Absolute error vs exact = {abs(vqe_energy - ground_energy_exact):.6e}')

## 6) (Optional) Convergence plot

If the Qiskit `result` object contains an optimization history, we attempt to extract and plot the energy per iteration. Different Qiskit versions store optimizer history differently; we try multiple common fields.

In [ ]:
# 6) Plot optimizer energy history if available
energy_history = None
try:
    # Newer Qiskit versions: result.optimal_parameters, result.optimizer_evals might exist
    if hasattr(result, 'optimizer_history') and result.optimizer_history is not None:
        # optimizer_history often is a list of dicts with 'energy' keys
        energy_history = [r['energy'] for r in result.optimizer_history if 'energy' in r]
    elif hasattr(result, 'optimizer_evals') and result.optimizer_evals is not None:
        # optimizer_evals might be list of tuples (params, energy)
        energy_history = [e for (_, e) in result.optimizer_evals]
except Exception:
    energy_history = None

if energy_history is not None and len(energy_history) > 0:
    plt.figure(figsize=(6,4))
    plt.plot(energy_history, marker='o')
    plt.xlabel('Iteration')
    plt.ylabel('Energy')
    plt.title('VQE energy convergence')
    plt.grid(True)
    plt.show()
else:
    print('\nNo optimizer energy history found in the returned result object to plot. This is common across Qiskit versions.')

## 7) Optional: prepare the variational state and compute fidelity with exact ground state

If the VQE result contains the optimal parameters, we can build the ansatz circuit with those parameters, simulate the statevector, and compute fidelity with the exact ground-state vector from diagonalization.

In [36]:
# 7) Extract optimal parameters and compute fidelity (if possible)
try:
    optimal_params = result.optimal_parameters
    print('\nOptimal variational parameters:')
    print(optimal_params)

    # Bind parameters and get statevector
    circ = ansatz.bind_parameters(optimal_params)
    circ = transpile(circ, backend=backend)
    circ.save_statevector()
    job = backend.run(circ)
    out = job.result()
    psi = out.get_statevector(circ)

    # exact ground state vector (from eigvecs)
    gs_index = np.argmin(eigvals)
    gs_vec = eigvecs[:, gs_index]

    fidelity = np.abs(np.vdot(gs_vec, psi))**2
    print(f'\nFidelity between VQE ansatz state and exact ground state = {fidelity:.12f}')
except Exception as e:
    print('\nCould not extract optimal parameters or compute fidelity; this may depend on your Qiskit version.')
    print('Error:', e)


Optimal variational parameters:
{ParameterVectorElement(θ[0]): np.float64(2.8491556753963945), ParameterVectorElement(θ[1]): np.float64(0.7559683168658696)}

Could not extract optimal parameters or compute fidelity; this may depend on your Qiskit version.
Error: 'RealAmplitudes' object has no attribute 'bind_parameters'


## 8) Notes and next steps

- If you want to run this on a noisy backend, we can add a noise model from Qiskit's Aer noise module.
- For multi-qubit Hamiltonians you'll typically map fermionic Hamiltonians (in chemistry) to qubit operators; then the structure is similar but requires more complex ansätze and optimizers.

---

If you want, I can also:

- add a noise-model section and run VQE under noise,
- export this notebook to a `.py` script or a downloadable `.ipynb` (done below),
- or merge this into the notebook you uploaded earlier.

In [35]:
# 1) Imports and basic setup
from qiskit_aer import AerSimulator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals

# Use StatevectorPrimitive instead
from qiskit.primitives import StatevectorEstimator

import numpy as np

# For reproducibility
seed = 42
algorithm_globals.random_seed = seed

# 2) Build the Hamiltonian
H_op = SparsePauliOp.from_list([
    ("Z", 2.0),
    ("X", 1.0), 
    ("I", 1.0)
])
print('Hamiltonian:')
print(H_op)

# 3) Exact classical diagonalization
H_mat = H_op.to_matrix()
eigvals, eigvecs = np.linalg.eigh(H_mat)
print('Exact eigenvalues:\n', eigvals)
ground_energy_exact = np.min(eigvals)
print(f'Exact ground-state energy = {ground_energy_exact:.12f}')

# 4) VQE setup - Use StatevectorEstimator
ansatz = RealAmplitudes(num_qubits=1, reps=1)
optimizer = COBYLA(maxiter=200)

# Use StatevectorEstimator for exact simulation
estimator = StatevectorEstimator()
vqe = VQE(estimator, ansatz, optimizer)

print('Ansatz:')
print(ansatz)

# 5) Run VQE
result = vqe.compute_minimum_eigenvalue(operator=H_op)
print('\nVQE result summary:')
print(result)
vqe_energy = result.eigenvalue.real
print(f'\nVQE estimated ground-state energy = {vqe_energy:.12f}')
print(f'Absolute error vs exact = {abs(vqe_energy - ground_energy_exact):.6e}')

Hamiltonian:
SparsePauliOp(['Z', 'X', 'I'],
              coeffs=[2.+0.j, 1.+0.j, 1.+0.j])
Exact eigenvalues:
 [-1.23606798  3.23606798]
Exact ground-state energy = -1.236067977500
Ansatz:
   ┌───────────────────────────┐
q: ┤ RealAmplitudes(θ[0],θ[1]) ├
   └───────────────────────────┘

VQE result summary:
{   'aux_operators_evaluated': None,
    'cost_function_evals': 34,
    'eigenvalue': np.float64(-1.2360679623853235),
    'optimal_circuit': <qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes object at 0x123b9f160>,
    'optimal_parameters': {   ParameterVectorElement(θ[0]): np.float64(2.8491556753963945),
                              ParameterVectorElement(θ[1]): np.float64(0.7559683168658696)},
    'optimal_point': array([2.84915568, 0.75596832]),
    'optimal_value': np.float64(-1.2360679623853235),
    'optimizer_evals': None,
    'optimizer_result': <qiskit_algorithms.optimizers.optimizer.OptimizerResult object at 0x123b9cc70>,
    'optimizer_time': 0.213852167129